# Defining Tools
Allow frontier models to connect with external functions
- richer responses by extending knowledge
- ability to carry out actions within the application
- enhanced capabilities, like calculations

How it works
- in a request to the LLM, specify available tools
- the reply is either text, or a request to run a tool
- we run the tool and call the llm with the results

## Common use cases for tools
Function calls can enable assistant to:
- fetch data or add knowledge or context
- take action, like booking a meeting
- perform calculations
- modify the UI

# Project - Airline AI Assistant
we'll now bring together what we've learned to make an AI Customer support assistant for an Airline.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

C:\Users\hp\.conda\envs\llms1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# initialization

load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
MODEL="gpt-4o-mini"
openai=OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [3]:
system_message="You are a helpful assistant for an Airline called FlightAI. "
system_message+="Give short, courteous answers, no more than 1 sentence. "
system_message+="Always be accurate. If you don't know the answer, say so."

In [6]:
def chat(message, history):
    messages=[{"role":"system", "content":system_message}]
    for human, assistant in history:
        messages.append({"role":"user", "content":human})
        messages.append({"role":"assistant", "content":assistant})
    messages.append({"role":"user", "content": message})
    response=openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [7]:
gr.ChatInterface(fn=chat).launch()

C:\Users\hp\.conda\envs\llms1\Lib\site-packages\gradio\components\chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7889

To create a public link, set `share=True` in `launch()`.


## Tools
Tools are an incredibly powerful features provided by the frontier LLMs.
with tools, you can write a function, and have the LLM call that function as part of its response.
sounds almost spoooky..we're giving it the power to run code on our machine?

In [8]:
# let's start by making a useful function.

ticket_prices={"london":"$799", "paris":"$899", "tokyo":"$1400", "berlin":"$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city=destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [9]:
get_ticket_price("PaRis")

Tool get_ticket_price called for PaRis


'$899'

In [11]:
# there's a particular dictionary structure that's required to describe our function:

price_function={
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [12]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

### Getting OpenAI to use our Tool
There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [14]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [15]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [16]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7890

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Paris
Tool get_ticket_price called for Berlin
Tool get_ticket_price called for Berlin
Tool get_ticket_price called for Berlin
Tool get_ticket_price called for Berlin


In [2]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
MODEL = "llama3.2:1b"
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [11]:
response= openai.chat.completions.create(
    model=MODEL,
    messages=[{"role":"system", "content":"You are a helpful Assistant."},
             {"role":"user", "content":"How many r is in 'strawberry'."}]
)

In [12]:
response.choices[0].message.content

'There are two Rs in the word "strawberry".'